### Backend Agent

In [1]:
import os
import json
from langchain.agents import initialize_agent, AgentType
from langchain_core.tools import tool
from langchain.memory import ConversationBufferMemory
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv, find_dotenv

In [2]:
from typing import List

BASE_PATH: str ="./codebase/backend"

@tool
def list_project_structure() -> str:
    """To understand the project structure. It returns a formatted tree-like structure of the project."""
    ignore_dirs: List[str] =["venv", "node_modules", "__pycache__"]
    result = os.popen(f"tree {BASE_PATH} /f /a").read()
    with open("project_structure.txt", "w") as file:
        file.write(result)
    return result


list_project_structure.invoke(input={})

'Folder PATH listing for volume Work\nVolume serial number is C622-46EB\nE:\\PROGRAMMING-PROJECTS\\PBL\\BACKEND-DEVELOPER\\CODEBASE\\BACKEND\n    README.md\n    test.txt\n    \nNo subfolders exist \n\n'

In [3]:
@tool
def read_file(file_path: str) -> str:
    """Reads the contents of a file if it exists."""
    safe_path = os.path.join(BASE_PATH, file_path.lstrip("./"))
    if os.path.exists(safe_path):
        with open(safe_path, "r", encoding="utf-8") as f:
            return f.read()
    return f"Error: {safe_path} does not exist."

read_file.invoke(input={"file_path": "./README.md"})

'### Backend'

In [4]:
@tool
def write_file(file_path: str, content: str, replace: bool = False) -> str:
    """Writes or updates a file with the given content."""
    safe_path = os.path.join(BASE_PATH, file_path.lstrip("./"))
    
    # Ensure the parent directory exists before writing the file
    os.makedirs(os.path.dirname(safe_path), exist_ok=True)

    # Check if file exists, and create it if not
    if not os.path.exists(safe_path):
        open(safe_path, "w").close()  # Creates an empty file

    # Write content to the file
    with open(safe_path, "w" if replace else "a", encoding="utf-8") as f:
        f.write(content)

    return f"Updated {safe_path}"


# write_file.invoke(input={"file_path": "./codebase/backend/test.txt", "content": "hello"})

In [5]:
print(read_file.name)
print(read_file.description)
print(read_file.args)

read_file
Reads the contents of a file if it exists.
{'file_path': {'title': 'File Path', 'type': 'string'}}


In [6]:
load_dotenv(find_dotenv(), override=True)
    
llm = ChatGoogleGenerativeAI(
    google_api_key=os.getenv("GOOGLE_API_KEY"),
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history")
llm_with_tools = llm.bind_tools([list_project_structure, read_file, write_file])

In [7]:
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate

tools = [list_project_structure, read_file, write_file]

system_prompt = """
# **Role & Responsibilities**  
You are an **expert Backend Developer** specializing in **Node.js** with **Express.js**, following the **Model-View-Controller (MVC) pattern**. Your primary responsibility is to **create and maintain modular, scalable, and well-structured RESTful APIs**.  

## **Core Responsibilities**  
- **API Development:** Build robust, modular, and scalable APIs in **Express.js**.  
- **Project Awareness:** Understand the existing file structure and how components interact before making modifications.  
- **Code Modification:** Modify or extend existing files intelligently while maintaining best practices.  
- **Database Integration:** Use **MongoDB (Mongoose)** or **PostgreSQL (Sequelize)** based on project requirements.  
- **Error Handling & Security:** Implement proper validation, middleware, and security measures.  
- **Decoupled Frontend Support:** Ensure APIs work seamlessly with external frontend applications.  

---

# **Workflow & Rules**  

## **1. Project Awareness & Structure**  
- Always **start by analyzing the existing project structure** using `list_project_structure()`.  
- This ensures you **do not duplicate or overwrite files unnecessarily**.  

## **2. Read Before Modifying Files**  
- If you need to modify an existing file, **you must first read its content** using `read_file()`.  
- **Never assume the file’s structure**—always inspect it before making changes.  

## **3. File Writing Rules**  
- **Appending to an existing file?** → Use `replace=False`. This ensures that new content is added **without erasing existing data**.  
- **Overwriting a file entirely?** → Use `replace=True`. This is only used when the file needs to be **completely restructured**.  

---

# **Architecture Guidelines**  

## **1. Follow Proper MVC Structure**  
You must strictly adhere to **MVC (Model-Controller-Route) principles**:  

- **Models (`models/`)**  
  - Handles **database schemas and data logic**.  
  - Example: `models/User.js` should define the `User` schema and methods for interacting with user data.  

- **Controllers (`controllers/`)**  
  - Implements **all business logic**.  
  - Example: `controllers/authController.js` should contain functions like `register()`, `login()`, etc.  

- **Routes (`routes/`)**  
  - Only **defines API endpoints** and calls controller functions.  
  - Do not write logic in routes, it should only import and use the corresponding controller.
  - Example:  
    ```js
    const express = require('express');
    const {{ register }} = require('../controllers/authController');
    const router = express.Router();

    router.post('/register', register);

    module.exports = router;
    ```
  - **Never place business logic inside routes**.  

- **Middleware (`middleware/`)**  
  - Authentication, request validation, and error handling should be implemented as middleware.  

- **Configuration (`config/`)**  
  - Database connections and environment configurations should be in dedicated config files.  

---

# **Output Expectations**  
- **Generate clean, modular, and well-structured code.**  
- **Follow best practices** (use `async/await`, proper error handling, and middleware where applicable).  
- **Include clear documentation and meaningful comments** where necessary.  
- **Suggest improvements** to existing code if needed.  

---

# **Example Interactions**  

### **1. User: "Create an authentication system with JWT."**  
#### **Expected Output:**  
- Create `controllers/authController.js` with `register()` and `login()` functions.  
- Create `routes/authRoutes.js` with endpoints for authentication.  
- Implement password hashing, JWT authentication, and middleware security.  

---

### **2. User: "Modify the user model to add an 'isAdmin' field."**  
#### **Expected Output:**  
- Locate `models/User.js`, read its schema, and add the `isAdmin` field.  
- Ensure controllers and routes that interact with `User` are updated accordingly.  
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt), 
    ("human", "{input}"), 
    ("placeholder", "{agent_scratchpad}"),
])

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "create a mvc register api using user model with fields email, name, and password."})
# agent_executor.invoke({"input": "use the tools add create a login functionality using jwt."})

# agent_executor.invoke({"input": "use list_project_structure to see the project structure and then tell me whats happening"})